In [1]:
# REQUIREMENTS FOR THIS PIPELINE
#  the recon output will be named e.g. "unedit.FS6_02". After manually editing
#  please rename this recon folder to "edit.FS6_02". Then the next time 
#  pipeline is run, it will re-submit the recon for this folder and create symlink.

#  the following notes need to updated periodically so this info can be piped:
#  files: LEADS_#####_date.csv (csv download file), Mayo_ADRIL_MRI_Quality_date.csv

        #details:
        #MOST IMPORTANT: once a new download has been initiated, load the download CSV file into
        #/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/
        #if it is a new image collection on loni, it will create a separate CSV file, 
        #otherwise will replace old one with concatenated data of that image collection.
        
        #must also download Mayo_ADRIL_MRI_Quality_date.csv and save to:
        #/autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY
        #this file will replace the old one (concatenates all data on loni about QC).

#  if any files need to be re-run / re-processed, delete the files in the 
#  folder. This pipeline does not overwrite anything.

# TO DO
#  send a recon job- just do for ones that do not have output?
#  not sure why but import_loni_notes is creates a new column in the dataframe scannotes?

In [2]:
# import modules
import io, os, sys, types # needed
import glob # needed
from nipype.pipeline.engine import Workflow, Node, MapNode # needed
from nipype.interfaces.utility import Function, IdentityInterface
import nipype.interfaces.io as nio
import nipype.pipeline.engine as pe
from nipype.interfaces.freesurfer import MRIConvert
from nipype.interfaces.freesurfer import ReconAll
from nipype import config
import pandas as pd
import re
import shutil
import pathlib
import pydicom
from pydicom.tag import Tag

In [3]:
# Clean and update spreadsheets

# these do not concatenate, must do this
downloadsdir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/'

# vertically concatenate all csvs
downloadlist = glob.glob(downloadsdir+'*.csv')
downloadlist.remove('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv')

# vertically concatenate all csvs
combined_csv = pd.concat( [ pd.read_csv(f) for f in downloadlist ] , sort=False)

# drop all non-MPRAGES, sort dataframe by subject column, drop all duplicates
combined_csv = combined_csv[combined_csv.Description == 'Accelerated Sagittal MPRAGE']
combined_csv = combined_csv.sort_values(by=['Downloaded'])
combined_csv = combined_csv.drop_duplicates(['Image Data ID'], keep='last')

# save combined download file
combined_csv.to_csv("/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv", index=False,)

# these download already concatenating all sessions ; use latest
qualitydir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY/'

list_of_qualityfiles = glob.glob(qualitydir+'*.csv')
MRIQUALITY = max(list_of_qualityfiles, key=os.path.getctime)

# make master list that pushes to scannotes or visa versa?

In [4]:
def scan(subject):
    subsessions = glob.glob(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/*/*/')
    repeat_tag = '-'
    for num in range(len(subsessions)):
        # look to see if more than one session on the same date
        # then look to see if more than one date (or both)
        parentfolder = subsessions[num].split('/')[8]
        filename = os.listdir(subsessions[num])[0] # dicom name to extract date
        #extract date from dicom:
        ds = pydicom.read_file(subsessions[num]+'/'+filename)
        date = str(ds[0x08, 0x22].value)
        #date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', filename).group()[4:]
        if num == len(subsessions)-1:
            namedate = dicomdir+subject+'_'+date
            os.rename(dicomdir+subject, namedate)
        else:
            namedateplus = dicomdir+subject+'_'+date+repeat_tag+'/Accelerated_Sagittal_MPRAGE/'+folder1
            pathlib.Path(namedateplus).mkdir(parents=True, exist_ok=True)
            shutil.move(subsessions[num], namedateplus)
            if not os.listdir(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/'+parentfolder):
                os.rmdir(dicomdir+subject+'/Accelerated_Sagittal_MPRAGE/'+parentfolder)
            repeat_tag = repeat_tag+'-'

In [5]:
# specify variables
leadsdir = '/cluster/animal/scan_data/leads/'
os.chdir(leadsdir)
dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon/unpack.log"
recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]
subjlist = [f for f in os.listdir(dicomdir) if (("_") not in f) and (not f.startswith('.') and ("duplicate" not in f))]

# wipe clean batch.recon.list
open(recondir+'batch.recon.list', 'w').close()

# unwravel all subjects with multiple sessions and rename to include date
for sub in subjlist:
    scan(sub)

# now just make a list of subjects by ID (this will be the input to the nodes)
# will define dicom path within node
sh_dicomlist = [f for f in os.listdir(dicomdir) if (("_" in f) and ("REPEAT_RUNS" not in f))]

# define workflow
leads_workflow = Workflow(name='leads_workflow') #, base_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/') # add  base_dir='/shared_fs/nipype_scratch'

# configure to stop on first crash
cfg = dict(execution={'stop_on_first_crash': True})
config.update_config(cfg)

In [6]:
# for debugging
sh_dicomlist = ['LDS3600056_20190323', 'LDS3600068_20190325']


In [7]:
# TEST NODE : PASSSWORDS

def credentials(): # combined with find_dicom
    import getpass
    USER = getpass.getuser()
    print('Please enter your PASSWORD for launchpad access: ')
    PASS= getpass.getpass()
    return USER, PASS

PASSWORDS = pe.Node(Function(input_names=["user", "pw"],
                         output_names=["USER","PASS"], # actual dicom (redundant to create unpacking node visualization)
                         function=credentials),
                        name='PASSWORDS')


In [8]:
# NODE : CREATEDIR
def createdir(val, USER, PASS):
    import os
    import re
    import glob
    import pydicom
    from pydicom.tag import Tag
    val = val.split('/')[-1]
    dicomdir = "/autofs/cluster/animal/scan_data/leads/LEADS/"
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/' 
    reconpath = recondir+val+'/'
    imgpath = reconpath+'/mri/orig/'
    dumplocation = imgpath+'001.mgz'
    subject = val.split('_')[0]
    #date = val.split('_')[1].replace("-","")  # take out any '-'
    MPRAGE_path = glob.glob(dicomdir+val+'/Accelerated_Sagittal_MPRAGE/*/*')[0]
    pickdicom = glob.glob(dicomdir+val+'/Accelerated_Sagittal_MPRAGE/*/*/*')[0]
    ds = pydicom.read_file(pickdicom) #(MPRAGE_path+'/'+pickdicom)
    date = str(ds[0x08, 0x22].value)
    sessionid = MPRAGE_path.split("/")[-1]
    try:
        os.makedirs(imgpath)
    except(FileExistsError):
        pass
    return reconpath, MPRAGE_path, pickdicom, dumplocation, recondir, USER, PASS, imgpath, date
        
CREATEDIR = pe.Node(Function(input_names=["val", "USER", "PASS"],
                         output_names=["createdir_out1","createdir_out2", "createdir_out3", "createdir_out4", "createdir_out5", "USER", "PASS", "createdir_out6", "date"], # actual dicom (redundant to create unpacking node visualization)
                         function=createdir),
                        name='CREATEDIR')

In [9]:
# pretend_df = pd.read_csv('/autofs/cluster/animal/scan_data/leads/recon_nip/LDS9410060_20190220/scannotes.csv')
# #pretend_df.loc[pretend_df.index[0], 'scan_notes'] = "some value"
# qc_pass = ''
# concat_comments = 'test'
# download_date = '44/44/44'
# pretend_df.loc[pretend_df.index[0], 'loni_overallpass'] = qc_pass
# pretend_df.loc[pretend_df.index[0], 'scan_notes'] = concat_comments
# pretend_df.loc[pretend_df.index[0], 'download_date'] = download_date

# pretend_df

# # Unnamed: 0 ??

In [10]:
# NODE : IMPORT_LONI_INFO

def import_loni_notes(dicomname, date):
    import pandas as pd
    import glob
    import os
    import re
    # download info
    download_df = pd.read_csv('/autofs/cluster/animal/scan_data/leads/spreadsheets/LONI_DOWNLOADS/combined_downloads.csv')
    recon_dir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    dicom = dicomname.split("/")[-1]
    subid = dicom.split("_")[1]
    #date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', dicomname).group()[4:]
    imageid = dicom.split("_")[12][1:-4]
    scannotes_df = pd.read_csv(recon_dir+subid+'_'+date+'/scannotes.csv')
    try:
        download_date = download_df.loc[download_df['Image Data ID'] == float(imageid), 'Downloaded'].values[0]
    except(IndexError):
        pass
    # loni notes
    qualitydir = '/autofs/cluster/animal/scan_data/leads/spreadsheets/MRIQUALITY/'
    list_qc_files = glob.glob(qualitydir+'*.csv')
    MRIQUALITY = max(list_qc_files, key=os.path.getctime)
    # these download already concatenating all sessions ; use latest
    quality_df = pd.read_csv(MRIQUALITY)
    try:
        loni_overallpass = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_overallpass'].values[0]
        if loni_overallpass == 1:
            qc_pass = '1'
        elif loni_overallpass == 4:
            qc_pass = '0'
        else:
            qc_pass = ''
        study_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_comments'].values[0]
        study_protocol_comment = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_protocol_comment'].values[0]
        protocol_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'protocol_comments'].values[0]
        series_comments = quality_df.loc[quality_df['loni_image'] == float(imageid), 'series_comments'].values[0]
        series_quality = quality_df.loc[quality_df['loni_image'] == float(imageid), 'series_quality'].values[0]# if 3, needs review; if 2 it is ok
        if series_quality == 2:
            s_quality = 'Scan quality is acceptable according to acquisition team. '
        elif series_quality == 3:
            s_quality = 'Scan quality is questionable according to acquisition team and needs review. '
        elif series_quality == 4:
            s_quality = 'Scan quality is poor according to acquisition team and needs review. '
        else:
            s_quality = 'No scan quality data recorded from acquisition team. '
        study_rescan_requested = quality_df.loc[quality_df['loni_image'] == float(imageid), 'study_rescan_requested'].values[0]
        if study_rescan_requested == 'TRUE':
            rescan_requested = ' Study rescan has been requested. '
        else:
            rescan_requested = '. No study rescans have been requested. '

        # delete duplicates within list, delete nans
        comments_list = [s_quality,str(study_comments),str(study_protocol_comment),str(series_comments),str(protocol_comments),rescan_requested]
        cleanedList = [x for x in comments_list if (x != 'nan')]
        concat_comments = ''.join(cleanedList)
    except(IndexError):
        concat_comments = 'No comments from acquisition team. '
        qc_pass = ''
    
    scannotes_df.loc[scannotes_df.index[0], 'loni_overallpass'] = qc_pass
    scannotes_df.loc[scannotes_df.index[0], 'scan_notes'] = concat_comments
    scannotes_df.loc[scannotes_df.index[0], 'download_date'] = download_date
    scannotes_df.to_csv(recon_dir+subid+'_'+date+'/scannotes.csv')
    
IMPORT_LONI_INFO = pe.Node(Function(input_names=["dicomname", "date"],
                         function=import_loni_notes),
                        name='IMPORT_LONI_INFO')


In [11]:
# NODE : UNPACK

def unpack(subjectdir, MPRAGE_path):
    from os import system
    import csv
    import pandas as pd
    import os.path
    if not os.path.isfile(subjectdir+'scan.info'): 
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_path, subjectdir, subjectdir)
        system(cmdstring)
    if not os.path.isfile(subjectdir+'scaninfo.csv'):
        with open(subjectdir+'/scan.info', 'r') as in_file:
            for line in in_file:
                editline = line.split()
                with open(subjectdir+'/scaninfo.csv', 'w') as result:
                    wr = csv.writer(result, dialect='excel')
                    wr.writerow(editline)
                result.close()
            in_file.close()
    scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
    subname = subjectdir.split('/')[-2]
    return subname, subjectdir, scan_info

UNPACK = pe.Node(Function(input_names=["subjectdir","MPRAGE_path"],
                         output_names=["unpack_out1","unpack_out2", "unpack_out3"], # actual dicom (redundant to create unpacking node visualization)
                         function=unpack),
                        name='UNPACK')


In [12]:
# NODE : CONVERT2MGZ
# I think I can set subject as variable to pass on here and others send elsewhere

CONVERT2MGZ = pe.Node(MRIConvert(out_type='mgz'), 
                        name='CONVERT2MGZ')

In [13]:
# # NODE : CONVERT2MGZ (only runs if .mgz is not available)

# def convert_dicom(in_file, out_file, imgpath):
#     import os
#     if os.path.isdir(imgpath):
#         if not os.path.exists(in_file):
#             cmdstring = 'mri_convert %s %s' % (in_file, out_file)
#             system(cmdstring)

# CONVERT2MGZ = pe.Node(Function(input_names=["in_file", "out_file", "imgpath"],
#                          output_names=["out_file"],
#                          function=convert_dicom),
#                         name='CONVERT2MGZ')

In [14]:
# NODE SCAN_AND_LOG
# note: decided to add this afterward precaution to increase efficiency because there are few errors
# and want to run the unpack and convert2mgz in parallel)

def scan_and_log(subjectdir, scan_info, mgz, reconfolder, subname):
    import re
    import os
    import pandas as pd
    # load in the scaninfo file
    dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
    check = scan_info.iloc[0,2] # first row (only row); second col (validity)
    if check != 'ok':
        with open(reconfolder+'/scanerrors', "a") as efile:
            efile.write(scan_info.iloc[0,7]) # log for errors in dicoms (or any ommitted scans)
    else:
        with open(reconfolder+'/batch.recon.list', "a") as bfile:
            bfile.write(subname)
        with open(reconfolder+'/unpack.log', "a") as ufile:
            ufile.write(subname)
        # should I makea scannotes? (will add info after recon)
        Elements = {'scan_notes': [''],'loni_overallpass': [''], 'download_date':[''], 'xnat_upload':[''], 'recon_path':[''],'recon_notes':[''], 'dickerson_overallpass':['']}
        df = pd.DataFrame(Elements, columns= ['scan_notes', 'loni_overallpass', 'download_date','xnat_upload','recon_path','recon_notes','dickerson_overallpass'])
        df.to_csv(subjectdir+'/scannotes.csv')
    return subjectdir, subname

SCAN_AND_LOG = pe.Node(Function(input_names=["subjectdir","scan_info",'mgz', 'reconfolder', 'subname'],
                         output_names=["subjectdir", "subname"],
                         function=scan_and_log),
                        name='SCAN_AND_LOG')

In [15]:
# NODE RECON_JOB

def recon_job(subjectname, USER, PASS): # add in username, pass, and subjectname
    # add condition :: run this only is FS_XX, or scripts does not exist!!
    import os
    import glob
    from paramiko import SSHClient
    reconpath = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    if ((glob.glob('/autofs/cluster/animal/scan_data/leads/recon_nip/'+subjectname+'/FS*') == []) or (glob.glob('/autofs/cluster/animal/scan_data/leads/recon_nip/'+subjectname+'/script*/') == [])):
        host="launchpad"
        user=USER
        pw=PASS
        client=SSHClient()
        client.load_system_host_keys()
        client.connect(host,username=user,password=pw, look_for_keys=False)
        tmpstr = '(cd /autofs/cluster/animal/scan_data/leads/analyses_nip; setenv p %s ; ./batch.recon.sh)' % subjectname
        stdin, stdout, stderr = client.exec_command(tmpstr)
        #stin = print("stdin: ", stdin.readlines())
        err = "stderr: ", stderr.readlines()
        out = "pwd: ", stdout.readlines()
        if len(err) < 1:
            warning = '0'
        else:
            warning = '1'
        return err, out, warning, subjectname

RECON_JOB = pe.Node(Function(input_names=["subjectname","USER", "PASS"], 
                        output_names=[ 'err', 'out', 'warning','subjectname'],
                         function=recon_job),
                        name='RECON_JOB')


In [16]:
# NODE: GATHER_FS_DETAILS (this part only after recon is done)

def gather_FS_details(subjectname): # add in username, pass, and subjectname
    import csv
    import os
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    # if you can access the status (complete) and 
    if os.path.isfile(recondir+subjectname+'/scripts/recon-all.done'): # replaces from old: recon-all.log
        recon_pending = 0
    #try:
        # obtain FS version
        versionfile = open(recondir+subjectname+'/scripts/build-stamp.txt', 'r')
        versionstring = versionfile.read()
        version = versionstring.split('-')
        result = [i for i in version if i.startswith('v')][0]
        long = result[1:]
        
        #obtain short verison of long
        size = len(long)
        x = 0
        while x ==0:
            if (long[-1] == '0') or (long[-1] == '.'): # shave off any . or 0s from the end of version number.
                long = long[:-1]
            else:
                x =1
        vlabel = 'FS'+long

        # obtain run number
        with open(recondir+subjectname+'/scaninfo.csv','r') as f:
            reader = csv.reader(f)
            scan_list = list(reader)
            runstring = scan_list[0][0] # run
            if len(runstring) == 1:
                runstring = '0'+runstring
        recon_name = vlabel+'_'+runstring
    else:
    #except(FileNotFoundError):
        recon_pending = 1
        print(subjectname+" Recon either already organized or not ready yet.")
        recon_name = ''
    return subjectname, recon_name, recon_pending
        
FS_DETAILS = pe.Node(Function(input_names=["subjectname"], 
                        output_names=[ 'subjectname', 'recon_name', 'recon_pending'],
                         function=gather_FS_details),
                        name='FS_DETAILS')

In [17]:
# NODE : MAKE_ORIG_FOLDER

def create_orig_folder(subjectname, recon_name, recon_pending):
    import os
    import shutil
    recondir = '/autofs/cluster/animal/scan_data/leads/recon_nip/'
    freesurfer_dirs = ['mri', 'stats', 'tmp', 'trash', 'touch', 'label', 'surf', 'scripts']
    if recon_pending == 0:
        # move all subfolders into this recon_name folder
        for fsdir in freesurfer_dirs:
            if os.path.isdir(recondir+subjectname+'/'+fsdir):
                shutil.move(recondir+subjectname+'/'+fsdir, recondir+subjectname+'/'+recon_name+'/'+fsdir)
    else:
        print(subjectname+" files moves already, or not prepared.")
    return subjectname, recon_name, recondir, recon_pending

MAKE_ORIGINAL_DIR = pe.Node(Function(input_names=["subjectname", "recon_name", "recon_pending"], 
                        output_names=['subjectname', 'recon_name', 'recondir', 'recon_pending'],
                         function=create_orig_folder),
                        name='MAKE_ORIGINAL_DIR')



In [18]:
# NODE : PREPARE_MANEDITS

def preparing_manedits(subjectname, recon_name, recondir, recon_pending):
    import shutil
    import pathlib
    import os
    analysesdir = '/autofs/cluster/animal/scan_data/leads/analyses_nip/'
    if recon_pending == 0: # otherwise dir already created or not ready)
        recon_name2 = 'unedit.'+recon_name
        #FS2 = pathlib.Path(recondir+subjectname+'/'+recon_name2).mkdir(parents=True, exist_ok=True)
        shutil.copytree(recondir+subjectname+'/'+recon_name, recondir+subjectname+'/'+recon_name2)
        # if edit.recon folder does not already exist, add to log
        with open(recondir+'to_edit.list\n', "a") as myfile:
            myfile.write(subjectname)
        
    # this will symlink before manual editing but ok since renaming unedit >> edit is required
    #Now create symlink to edit. in analysis folder!!
#     try:
#         os.symlink(recondir+subjectname+'/'+recon_name2, analysesdir+sub)
#     except(FileExistsError):
#         print(subjectname+" in analyses is already linked to unedit.recon.")
    return subjectname
        
PREPARE_MANEDITS = pe.Node(Function(input_names=['subjectname', 'recon_name', 'recondir', 'recon_pending'], 
                        output_names=['subjectname'],
                         function=preparing_manedits),
                        name='PREPARE_MANEDITS')

In [19]:
# NODE : obtain post-edit notes , FS version, push notes to xnat and update xnat_upload field




In [20]:
# # # NODE : INFOSOURCE
INFOSOURCE = Node(IdentityInterface(fields=['subject_name'], mandatory_inputs=False),
                  name="INFOSOURCE")

INFOSOURCE.iterables = ('subject_name', sh_dicomlist)

# NODE : SELECTFILES
#templates = dict(dicom=sh_dicomlist[0])    ## THIS WORKED!
templates = {
    "dicom": "{subject_name}" 
    }
SELECTFILES = Node(nio.SelectFiles(templates, base_directory=dicomdir),
                   name="SELECTFILES")

# NODE : DATASINK
DATASINK = Node(nio.DataSink(base_directory=leadsdir,
                container='recon_nip'),
                name="DATASINK")

In [21]:
# Connect all nodes (including INFOSOURCE, SELECTFILES, and DATASINK) to workflow

leads_workflow.connect([(INFOSOURCE, SELECTFILES, [('subject_name', 'subject_name')]),
                (SELECTFILES, CREATEDIR, [('dicom', 'val')]),
                (PASSWORDS, CREATEDIR, [('USER', 'USER')]),
                (PASSWORDS, CREATEDIR, [('PASS', 'PASS')]), 
                (CREATEDIR, IMPORT_LONI_INFO, [('date', 'date')]),
                (CREATEDIR, IMPORT_LONI_INFO, [('createdir_out3', 'dicomname')]),
                (CREATEDIR, UNPACK, [('createdir_out1', 'subjectdir')]),
                 (CREATEDIR, UNPACK, [('createdir_out2', 'MPRAGE_path')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out3', 'in_file')]),
                 (CREATEDIR, CONVERT2MGZ, [('createdir_out4', 'out_file')]),
                #(CREATEDIR, CONVERT2MGZ, [('createdir_out6', 'imgpath')]),
                (CONVERT2MGZ, SCAN_AND_LOG, [('out_file', 'mgz')]),
                (CREATEDIR, SCAN_AND_LOG, [('createdir_out5', 'reconfolder')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out1', 'subname')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out2', 'subjectdir')]),
                (UNPACK, SCAN_AND_LOG, [('unpack_out3', 'scan_info')]),
                (CREATEDIR, RECON_JOB, [('USER','USER')]),
                (CREATEDIR, RECON_JOB, [('PASS','PASS')]),
                (SCAN_AND_LOG, RECON_JOB, [('subname','subjectname')]), 
                (RECON_JOB, FS_DETAILS, [('subjectname','subjectname')]),
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('subjectname','subjectname')]), 
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('recon_pending','recon_pending')]), 
                (FS_DETAILS, MAKE_ORIGINAL_DIR, [('recon_name','recon_name')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('subjectname','subjectname')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recon_name','recon_name')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recondir','recondir')]), 
                (MAKE_ORIGINAL_DIR, PREPARE_MANEDITS, [('recon_pending','recon_pending')]), 
                (PREPARE_MANEDITS, DATASINK, [('subjectname','backup')])  # backup folder?
                 ])

In [22]:
# Execute your workflow in sequential way
# leads_workflow.run(run(plugin='MultiProc', plugin_args={'n_procs' : 2})
leads_workflow.run()

leads_workflow.write_graph(graph2use='flat')

190327-09:55:49,29 nipype.workflow INFO:
	 Workflow leads_workflow settings: ['check', 'execution', 'logging', 'monitoring']
190327-09:55:49,94 nipype.workflow INFO:
	 Running serially.
190327-09:55:49,96 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmpi6p2oo2s/leads_workflow/_subject_name_LDS3600068_20190325/SELECTFILES".
190327-09:55:49,103 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190327-09:55:49,112 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190327-09:55:49,114 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.SELECTFILES" in "/tmp/tmp1_spplpb/leads_workflow/_subject_name_LDS3600056_20190323/SELECTFILES".
190327-09:55:49,120 nipype.workflow INFO:
	 [Node] Running "SELECTFILES" ("nipype.interfaces.io.SelectFiles")
190327-09:55:49,126 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.SELECTFILES".
190327-09:55:49,127 nipype.workflow INFO:
	 [Node] Setting-up "le

190327-09:56:16,177 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	RunNo             1
190327-09:56:16,179 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	SeriesNo          2
190327-09:56:16,180 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	ImageNo           1
190327-09:56:16,181 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	NImageRows        256
190327-09:56:16,182 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	NImageCols        240
190327-09:56:16,183 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	NFrames           1
190327-09:56:16,184 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	SliceArraylSize   1
190327-09:56:16,185 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	IsMosaic          0
190327-09:56:16,186 nipype.interface INFO:
	 stdout 2019-03-27T09:56:16.150673:	ImgPos            123.8390 139.6610 141.1186 
190327-09:56:16,187 nipype.interface INFO:
	 stdout 2019-03-27T09:56:1

190327-09:56:55,83 nipype.interface INFO:
	 stderr 2019-03-27T09:56:55.081845:INFO: Scanning for Series Number 12
190327-09:56:56,101 nipype.interface INFO:
	 stdout 2019-03-27T09:56:56.101430:INFO: loading series header info.
190327-09:56:56,103 nipype.interface INFO:
	 stderr 2019-03-27T09:56:56.103057:INFO: found 208 files in series
190327-09:57:16,683 nipype.interface INFO:
	 stderr 2019-03-27T09:57:16.683443:
190327-09:57:16,684 nipype.interface INFO:
	 stdout 2019-03-27T09:57:16.684695:INFO: sorting.
190327-09:57:16,685 nipype.interface INFO:
	 stdout 2019-03-27T09:57:16.684695:sdfiSameSlicePos() eps = 0.000001
190327-09:57:16,686 nipype.interface INFO:
	 stdout 2019-03-27T09:57:16.684695:INFO: (240 256 208), nframes = 1, ismosaic=0
190327-09:57:16,687 nipype.interface INFO:
	 stderr 2019-03-27T09:57:16.687477:RunNo = 11
190327-09:57:16,691 nipype.interface INFO:
	 stdout 2019-03-27T09:57:16.691168:sdfi->UseSliceScaleFactor 0
190327-09:57:16,692 nipype.interface INFO:
	 stdout 20

190327-09:57:20,839 nipype.workflow INFO:
	 [Node] Running "RECON_JOB" ("nipype.interfaces.utility.wrappers.Function")
190327-09:57:32,451 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.RECON_JOB".
190327-09:57:32,454 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.FS_DETAILS" in "/tmp/tmp4kflo1n6/leads_workflow/_subject_name_LDS3600056_20190323/FS_DETAILS".
190327-09:57:32,463 nipype.workflow INFO:
	 [Node] Running "FS_DETAILS" ("nipype.interfaces.utility.wrappers.Function")
LDS3600056_20190323 Recon either already organized or not ready yet.
190327-09:57:32,476 nipype.workflow INFO:
	 [Node] Finished "leads_workflow.FS_DETAILS".
190327-09:57:32,477 nipype.workflow INFO:
	 [Node] Setting-up "leads_workflow.MAKE_ORIGINAL_DIR" in "/tmp/tmppwyoaq9k/leads_workflow/_subject_name_LDS3600056_20190323/MAKE_ORIGINAL_DIR".
190327-09:57:32,486 nipype.workflow INFO:
	 [Node] Running "MAKE_ORIGINAL_DIR" ("nipype.interfaces.utility.wrappers.Function")
190327-09:57:32,497 nipype.w

RuntimeError: Workflow did not execute cleanly. Check log for details

In [ ]:
leads_workflow.write_graph(graph2use='flat')

In [ ]:
subjectname = 'LDS3600068_20190325'
glob.glob('/autofs/cluster/animal/scan_data/leads/recon_nip/'+subjectname+'/script*/') == []